# Resistance/Support AND Candles Patterns

https://www.youtube.com/watch?v=vPBwO6Fa6rc&list=TLPQMjgwMTIwMjR38oWwJE0Wmw&index=1

In [1]:
import pandas as pd
import os
import pickle


from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
print(BASE_DIR)

from scripts.backtesting import Strategy
from scripts.backtesting import Backtest


c:\Users\lbisa\Dropbox\Cripto\python\InvestingEvolution\ie\www\backtest
C:\Users\lbisa\Dropbox\Cripto\python\InvestingEvolution\ie\www


c:\Users\lbisa\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
klines_file =  './backtest/klines/2d01/Completo_BTCUSDT_2d01_2021-01-01_2023-07-31.DataFrame'
#klines_file = './backtest/klines/2d01/Lateral_BTCUSDT_2d01_2023-06-20_2023-10-09.DataFrame'
#klines_file = './backtest/klines/1h01/Completo_BTCUSDT_1h01_2021-01-01_2023-07-31.DataFrame'
#klines_file = './backtest/klines/1h01/Lateral_BTCUSDT_1h01_2023-06-20_2023-10-09.DataFrame'
#klines_file = './backtest/klines/1h01/Bajista_BTCUSDT_1h01_2021-04-12_2021-07-12.DataFrame'
#klines_file = './backtest/klines/1h01/Alcista_BTCUSDT_1h01_2020-09-07_2021-05-03.DataFrame'
#klines_file = './backtest/klines/0m15/Completo_BTCUSDT_0m15_2021-01-01_2023-07-31.DataFrame'
#klines_file = './backtest/klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame'

with open(klines_file, 'rb') as file:
    df = pickle.load(file)

start_date = klines_file.split('/')[4].split('_')[3] #Extrae la fecha de inicio
start_date = pd.to_datetime(start_date, format='%Y-%m-%d')
start_date = start_date.replace(hour=0, minute=0, second=0)
df = df[df['datetime']>=start_date]
df.reset_index(inplace=True)
df.drop('index',axis=1, inplace=True)
df.rename(columns={'datetime': 'Local time'}, inplace=True)
df

,Local time,open,high,low,close,volume
0,2021-01-01,28923.63,29600.00,28624.57,29331.69,54182.925011
1,2021-01-02,29331.70,33300.00,28946.53,32178.33,129993.873362
2,2021-01-03,32176.45,34778.11,31962.99,33000.05,120957.566750
3,2021-01-04,33000.05,33600.00,28130.00,31988.71,140899.885690
4,2021-01-05,31989.75,34360.00,29900.00,33949.53,116049.997038
...,...,...,...,...,...,...
938,2023-07-28,29222.78,29542.22,29123.12,29314.14,23993.616270
939,2023-07-29,29314.14,29406.92,29256.18,29352.90,10851.368440
940,2023-07-30,29352.90,29449.00,29033.24,29281.09,15706.974410
941,2023-07-31,29281.09,29530.00,29101.80,29232.25,22605.489640


# Support and Resistance FUNCTIONS

In [3]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.low[i]>df1.low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.low[i]<df1.low[i-1]):
            return 0
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.high[i]<df1.high[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.high[i]>df1.high[i-1]):
            return 0
    return 1

In [4]:
length = len(df)
high = list(df['high'])
low = list(df['low'])
close = list(df['close'])
open = list(df['open'])
bodydiff = [0] * length

highdiff = [0] * length
lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
       
def isStar(l):
    bodydiffmin = 0.0020
    row=l
    highdiff[row] = high[row]-max(open[row],close[row])
    lowdiff[row] = min(open[row],close[row])-low[row]
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001
    ratio1[row] = highdiff[row]/bodydiff[row]
    ratio2[row] = lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and lowdiff[row]<0.2*highdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]>close[row]):
        return 1
    elif (ratio2[row]>1 and highdiff[row]<0.2*lowdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]<close[row]):
        return 2
    else:
        return 0
    
def closeResistance(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.high[l]-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c2 = abs(max(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c3 = min(df.open[l],df.close[l])<min(levels, key=lambda x:abs(x-df.high[l]))
    c4 = df.low[l]<min(levels, key=lambda x:abs(x-df.high[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0
    
def closeSupport(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.low[l]-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c2 = abs(min(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c3 = max(df.open[l],df.close[l])>min(levels, key=lambda x:abs(x-df.low[l]))
    c4 = df.high[l]>min(levels, key=lambda x:abs(x-df.low[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0

In [5]:
n1=2
n2=2
backCandles=30
signal = [0] * length

for row in range(backCandles, len(df)-n2):
    ss = []
    rr = []
    for subrow in range(row-backCandles+n1, row+1):
        if support(df, subrow, n1, n2):
            ss.append(df.low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.high[subrow])
    #!!!! parameters
    if ((isEngulfing(row)==1 or isStar(row)==1) and closeResistance(row, rr, 150e-5) ):#and df.RSI[row]<30
        signal[row] = 1
    elif((isEngulfing(row)==2 or isStar(row)==2) and closeSupport(row, ss, 150e-5)):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0


In [6]:
df['signal']=signal
df.columns = ['Local time', 'Open', 'High', 'Low', 'Close', 'Volume', 'signal']
def SIGNAL():
    return df.signal

In [7]:
class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1==2:
            usd = self._broker.equity
            size = (usd/self._broker.last_price)*0.99
            sl1 = self.data.Close[-1] * 0.985
            tp1 = self.data.Close[-1] * 1.03
            self.buy(size=size,sl=sl1, tp=tp1)
        #elif self.signal1==1:
        #    sl1 = self.data.Close[-1] * 1.03
        #    tp1 = self.data.Close[-1] * 0.97
        #    self.sell(size=0.2,sl=sl1, tp=tp1)
            
bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.002)
stat = bt.run()
bt.plot()
stat

C:\Users\lbisa\AppData\Local\Temp\ipykernel_12984\316850432.py:19: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.002)


Start                                     0.0
End                                     942.0
Duration                                942.0
Exposure Time [%]                    2.969247
Equity Final [$]                 10896.284813
Equity Peak [$]                  11396.496501
Return [%]                           8.962848
Buy & Hold Return [%]                1.276094
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.389171
Avg. Drawdown [%]                   -2.940706
Max. Drawdown Duration                  227.0
Avg. Drawdown Duration                  184.0
# Trades                                 12.0
Win Rate [%]                             50.0
Best Trade [%]                            3.0
Worst Trade [%]                          -1.5
Avg. Trade [%]                    

In [10]:
stat._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,USDBuy,USDSell,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
0,0.173987,74,75,56900.75,56047.23875,9899.999894,9751.499896,-148.499998,-0.015,74,75,None,1
1,0.161004,300,301,60575.89,62393.16670,9752.984824,10045.574369,292.589545,0.030,300,301,None,1
2,0.269156,398,399,37311.61,38430.95830,10042.648552,10343.928008,301.279457,0.030,398,399,None,1
3,0.253447,472,474,40801.13,42025.16390,10340.915427,10651.142890,310.227463,0.030,472,474,None,2
4,0.471492,530,531,22583.72,22244.96420,10648.040600,10488.319991,-159.720609,-0.015,530,531,None,1
5,0.526016,545,546,19942.21,19643.07685,10489.917148,10332.568391,-157.348757,-0.015,545,546,None,1
6,0.486206,571,572,21254.67,21892.31010,10334.141918,10644.166176,310.024258,0.030,571,572,None,1
7,0.618313,710,711,17209.83,17726.12490,10641.065919,10960.297897,319.231978,0.030,710,711,None,1
8,0.473165,785,787,23157.07,23851.78210,10957.105532,11285.818698,328.713166,0.030,785,787,None,2
9,0.559908,798,799,20150.69,19848.42965,11282.531529,11113.293556,-169.237973,-0.015,798,799,None,1
